In [4]:
%pip install tensorflow pandas matplotlib

Note: you may need to restart the kernel to use updated packages.


In [7]:
import sys
print("Python version:", sys.version)
print("Python interpreter path:", sys.executable)

Python version: 3.9.20 (main, Oct  3 2024, 07:27:41) 
[GCC 11.2.0]
Python interpreter path: /vol/home/s4422090/miniconda3/envs/myenv/bin/python


In [1]:
!nvidia-smi

Sat Nov  2 21:21:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4060        Off | 00000000:01:00.0 Off |                  N/A |
| 32%   31C    P8              N/A / 115W |     69MiB /  8188MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

conda create --name IDL python=3.12 jupyter 

conda activate IDL


pip install tensorflow[and-cuda]

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2024-11-02 21:21:17.849059: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-02 21:21:17.851214: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-02 21:21:17.871562: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-02 21:21:17.872098: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-02 21:21:20.995574: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

Num GPUs Available:  0


2024-11-02 21:21:28.041695: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-02 21:21:28.042384: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth set for GPUs")
    except RuntimeError as e:
        print(e)

In [ ]:
pip install tensorflow keras numpy requests scikit-learn matplotlib ConfigSpace

In [1]:
import numpy as np 
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.layers import Lambda


def load_and_split_data(images_path, labels_path, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1, random_state=42):
    images = np.load(images_path)
    labels = np.load(labels_path) 

    # Normalize the images to [0,1]
    images = images.astype('float32') / 255.0 
    

    num_samples, height, width = images.shape

    # Shuffle + separate into training, validation, test
    X_train, X_temp, y_train, y_temp = train_test_split(
        images, labels, 
        test_size= (1 - train_ratio), 
        random_state=random_state, 
        shuffle=True
    )

    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, 
        test_size= val_ratio / (test_ratio + val_ratio), 
        random_state=random_state, 
        shuffle=True
    )   
    
    return [X_train, y_train, X_val, y_val, X_test, y_test]

def label_transformation(labels):

    hours = labels[:, 0].astype(float)
    minutes = labels[:, 1].astype(float)

    # Convert time to continuous value: hour + minutes/60
    continuous_labels = hours + minutes / 60.0
    return continuous_labels

def square_time_loss_function(y_true, y_pred):
    diff = tf.abs(y_true - y_pred)
    circular_diff = tf.minimum(diff, 12 - diff)
    return tf.square(circular_diff)

def absolute_time_loss_function(y_true, y_pred):
    diff = tf.abs(y_true - y_pred)
    circular_diff = tf.minimum(diff, 12 - diff)
    return circular_diff

if __name__ == "__main__":
    images_path = "data/images.npy"
    labels_path = 'labels.npy'
    
    X_train, y_train, X_val, y_val, X_test, y_test = load_and_split_data(images_path=images_path, labels_path=labels_path)


    # Transform the labels to continuous values
    y_test_cont = label_transformation(y_test)
    y_train_cont = label_transformation(y_train)
    y_val_cont = label_transformation(y_val)



    width, height = X_train.shape[1], X_train.shape[2]
    input_shape = (width, height, 1)

    model = Sequential([
        Conv2D(75, kernel_size=(3, 3), activation='relu', input_shape=input_shape),

        Conv2D(150, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),  
            
        Conv2D(300, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25), 

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),  
        Dense(1)  # Output layer for regression, single continuous output
    ]) 


    model.compile(
        loss= square_time_loss_function,  
        optimizer='adam',                    
        metrics=[absolute_time_loss_function]  # Mean Absolute Error as an additional metric
    )

    model.fit(
        X_train, y_train_cont,
        batch_size=128,
        epochs=10,
        verbose=1,
        validation_data=(X_val, y_val_cont)   
    )

    # Evaluate the model
    score = model.evaluate(X_test, y_test_cont, verbose=0)
    print('Test loss:', score[0])
    print('Test mean minute error', score[1]*60)
    print('Test mean minute error', score[1])


2024-11-04 13:41:05.412025: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-04 13:41:06.347841: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-04 13:41:11.620611: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-04 13:41:11.661699: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


KeyboardInterrupt: 